In [1]:
import jax
import jax.numpy as np
import jraph
import flax.linen as nn

from functools import partial



In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

from models.gnn import GraphConvNet
from models.graph_utils import nearest_neighbors

2023-03-10 14:48:38.869017: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /n/helmod/apps/centos7/Core/glib/2.56.1-fasrc01/lib64:/n/helmod/apps/centos7/Core/gcc/12.1.0-fasrc01/lib64:/n/helmod/apps/centos7/Core/gcc/12.1.0-fasrc01/lib:/n/helmod/apps/centos7/Core/cudnn/8.5.0.96_cuda11-fasrc01/lib:/n/helmod/apps/centos7/Core/cuda/11.7.1-fasrc01/cuda/extras/CUPTI/lib64:/n/helmod/apps/centos7/Core/cuda/11.7.1-fasrc01/cuda/lib64:/n/helmod/apps/centos7/Core/cuda/11.7.1-fasrc01/cuda/lib:/n/helmod/apps/centos7/Core/mpc/1.2.1-fasrc01/lib64:/n/helmod/apps/centos7/Core/mpfr/4.1.0-fasrc01/lib64:/n/helmod/apps/centos7/Core/gmp/6.2.1-fasrc01/lib64
2023-03-10 14:48:38.869336: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvin

In [3]:
n_nodes = 5000
x = np.load("/n/holyscratch01/iaifi_lab/ccuesta/data_for_sid/halos.npy")[:, :n_nodes,:]
#x = np.load("../data/nbody_samples_only_pos.npz")['x_true'][:, :n_nodes, :]

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
sources, targets = jax.vmap(nearest_neighbors, in_axes=(0, None))(x[:4], 20)

In [5]:
graph = jraph.GraphsTuple(
          n_node=np.array(4 * [[n_nodes]]), 
          n_edge=np.array(4 * [[10]]),
          nodes=x[:4], 
          edges=None,#np.zeros(sources.shape),
          globals=np.ones((4, 7)),
          senders=sources,
          receivers=targets)

In [6]:
class GraphWrapper(nn.Module):
    @nn.compact
    def __call__(self, x):
        model = jax.vmap(GraphConvNet(
            latent_size=6, 
            num_mlp_layers=4, 
            message_passing_steps=3, 
            skip_connections=False))
        return model(x)

In [7]:
model = GraphWrapper()
rng = jax.random.PRNGKey(42)
graph_out, _ = model.init_with_output(rng, graph)

First globals
(7,)
Second globals
(1, 7)
Edge Func
edges =  Traced<ShapedArray(float32[100000])>with<BatchTrace(level=1/0)> with
  val = Array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
  batch_dim = 0
senders =  (100000, 6)
receivers =  (100000, 6)
globals =  (100000, 7)


ValueError: axis 1 is out of bounds for array of dimension 1

In [ ]:
graph_out.nodes.shape

NameError: name 'graph_out' is not defined

In [ ]:
import sys
sys.path.append("../")

from models.diffusion import ScoreNet

In [ ]:
model = ScoreNet(5, 16)
rng = jax.random.PRNGKey(42)

n_batch = 4

_, _ = model.init_with_output(rng, x[:n_batch], np.linspace(0., 1., n_batch), np.ones((n_batch, 6)), np.ones((n_batch, x.shape[1])))

In [11]:
sources, targets = jax.vmap(nearest_neighbors, in_axes=(0, None))(x[:n_batch], 20, mask=np.ones((n_batch, x.shape[1])))